In [405]:
import json
import gzip
import math
import numpy
from collections import defaultdict
from sklearn import linear_model
import random
import statistics
import numpy as np

In [406]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [407]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [408]:
answers = {}

In [409]:
# From https://cseweb.ucsd.edu/classes/fa24/cse258-b/files/steam.json.gz
z = gzip.open("drive/MyDrive/CSE 258/steam.json.gz")

In [410]:
dataset = []
for l in z:
    d = eval(l)
    dataset.append(d)

In [411]:
z.close()

In [412]:
dataset[1]

{'userID': 'u70666506',
 'early_access': False,
 'hours': 63.5,
 'hours_transformed': 6.011227255423254,
 'found_funny': 1,
 'text': 'If you want to sit in queue for 10-20min and have 140 ping then this game is perfect for you :)',
 'gameID': 'g49368897',
 'user_id': '76561198030408772',
 'date': '2017-05-20'}

In [413]:
### Question 1

In [414]:
from sklearn.metrics import mean_squared_error
def MSE(y, ypred):
    return mean_squared_error(y, ypred)

In [415]:
X = []
y = []
for d in dataset:
  X.append(len(d['text']))
  y.append(d['hours'])
X = np.array(X).reshape(-1, 1)

In [416]:
mod = linear_model.LinearRegression().fit(X,y)
y_pred = mod.predict(X)
mse1 = MSE(y,y_pred)

In [418]:
mod.intercept_

66.00069845208783

In [419]:
answers['Q1'] = [float(mod.coef_[0]), float(mse1)] # Remember to cast things to float rather than (e.g.) np.float64

In [421]:
assertFloatList(answers['Q1'], 2)

In [ ]:
### Question 2

In [422]:
dataTrain = dataset[:int(len(dataset)*0.8)]
dataTest = dataset[int(len(dataset)*0.8):]
lenTrain = len(dataTrain)
lenTest = len(dataTest)

In [423]:
X_train = X[:int(len(dataset)*0.8)]
y_train = y[:int(len(dataset)*0.8)]
X_test = X[int(len(dataset)*0.8):]
y_test = y[int(len(dataset)*0.8):]
X_train = np.array(X_train).reshape(-1, 1)
X_test = np.array(X_test).reshape(-1, 1)

In [424]:
mod = linear_model.LinearRegression().fit(X_train,y_train)
y_pred = mod.predict(X_test)
mse2 = MSE(y_test,y_pred)

In [426]:
under = 0
over = 0
for i in range(lenTest):
  if y_pred[i] < y_test[i]:
    under += 1
  elif y_pred[i] > y_test[i]:
    over += 1

In [427]:
answers['Q2'] = [mse2, under, over]

In [429]:
assertFloatList(answers['Q2'], 3)

In [ ]:
### Question 3

In [430]:
y2 = y_train
sorted_y2 = sorted(enumerate(y2), key=lambda x: x[1], reverse=False)
perc90 = []
for i in range(int(lenTrain*0.9)):
  perc90.append(sorted_y2[i][0])
X3a = []
y3a = []
for i in perc90:
  X3a.append(X_train[i])
  y3a.append(y_train[i])
X3a = np.array(X3a).reshape(-1, 1)

mod3a = linear_model.LinearRegression()
# mod3a = linear_model.LinearRegression()
mod3a.fit(X3a,y3a)
pred3a = mod3a.predict(X_test)

In [431]:
under3a = 0
over3a = 0
for i in range(lenTest):
  if pred3a[i] < y_test[i]:
    under3a += 1
  elif pred3a[i] > y_test[i]:
    over3a += 1

In [432]:
y3b = []
y3b_test_transformed = []
for i in range(lenTrain):
  y3b.append(dataTrain[i]['hours_transformed'])
for i in range(lenTest):
  y3b_test_transformed.append(dataTest[i]['hours_transformed'])

mod3b = linear_model.LinearRegression()
mod3b.fit(X_train,y3b)
pred3b = mod3b.predict(X_test)

In [433]:
under3b = 0
over3b = 0
for i in range(lenTest):
  if pred3b[i] < y3b_test_transformed[i]:
    under3b += 1
  elif pred3b[i] > y3b_test_transformed[i]:
    over3b += 1

In [434]:
mod.intercept_

65.94674890675668

In [461]:
theta0 = mod.intercept_
x3 = sorted(X_train)
median_length = (x3[int((lenTrain-1)/2)]+x3[int(lenTrain/2)])/2
y3 = sorted(y_train)
median_hours = (y3[int((lenTrain-1)/2)]+y3[int(lenTrain/2)])/2
theta1 = (median_hours-theta0)/median_length

In [468]:
pred3c = theta0 + theta1 * X_test

In [470]:
under3c = 0
over3c = 0
for i in range(lenTest):
  if pred3c[i] < y_test[i]:
    under3c += 1
  elif pred3c[i] > y_test[i]:
    over3c += 1

In [ ]:
# etc. for 3b and 3c

In [471]:
answers['Q3'] = [under3a, over3a, under3b, over3b, under3c, over3c]

In [145]:
assertFloatList(answers['Q3'], 6)

In [ ]:
### Question 4

In [475]:
y_train4 = []
for y in y_train:
  y_train4.append(y>median_hours)
y_test4 = []
for y in y_test:
  y_test4.append(y>median_hours)

In [476]:
mod = linear_model.LogisticRegression(C=1)
mod.fit(X_train,y_train4)
predictions = mod.predict(X_test) # Binary vector of predictions

In [477]:
def ber(pred,test):
  TP = 0
  TN = 0
  FP = 0
  FN = 0
  for i in range(len(pred)):
    if pred[i] == test[i]:
      if test[i] == True:
        TP += 1
      else:
        TN += 1
    else:
      if test[i] == True:
        FN += 1
      else:
        FP += 1

  BER = (FN/(TP+FN) + FP/(FP+TN)) / 2
  return TP, TN, FP, FN, BER

In [478]:
TP, TN, FP, FN, BER = ber(predictions,y_test4)

In [479]:
answers['Q4'] = TP, TN, FP, FN, BER

In [481]:
assertFloatList(answers['Q4'], 5)

In [ ]:
### Question 5

In [482]:
answers['Q5'] = FP, FN

In [268]:
assertFloatList(answers['Q5'], 2)

In [ ]:
### Question 6

In [275]:
X2014 = []
y2014 = []
X2015 = []
y2015 = []

for d in dataTrain:
  if int(d['date'][:4])<2015:
    X2014.append(len(d['text']))
    y2014.append(d['hours']>median_hours)
  else:
    X2015.append(len(d['text']))
    y2015.append(d['hours']>median_hours)

X2014test = []
y2014test = []
X2015test = []
y2015test = []

for d in dataTest:
  if int(d['date'][:4])<2015:
    X2014test.append(len(d['text']))
    y2014test.append(d['hours']>median_hours)
  else:
    X2015test.append(len(d['text']))
    y2015test.append(d['hours']>median_hours)

X2014 = np.array(X2014).reshape(-1, 1)
X2015 = np.array(X2015).reshape(-1, 1)
X2014test = np.array(X2014test).reshape(-1, 1)
X2015test = np.array(X2015test).reshape(-1, 1)

In [276]:
mod = linear_model.LogisticRegression(C=1)
mod.fit(X_train,y_train4)
y_pred2014 = mod.predict(X2014test)
y_pred2015 = mod.predict(X2015test)

In [277]:
modC = linear_model.LogisticRegression(C=1)
mod.fit(X2014,y2014)
y_pred2015_ = mod.predict(X2015test)

In [278]:
modD = linear_model.LogisticRegression(C=1)
mod.fit(X2015,y2015)
y_pred2014_ = mod.predict(X2014test)

In [284]:
BER_A = ber(y_pred2014,y2014test)[4]
BER_B = ber(y_pred2015,y2015test)[4]
BER_C = ber(y_pred2015_,y2015test)[4]
BER_D = ber(y_pred2014_,y2014test)[4]

In [285]:
answers['Q6'] = [BER_A, BER_B, BER_C, BER_D]

In [287]:
assertFloatList(answers['Q6'], 4)

In [ ]:
### Question 7

In [372]:
usersPerItem = defaultdict(set) # Maps an item to the users who rated it
itemsPerUser = defaultdict(set) # Maps a user to the items that they rated
hoursPerUser = defaultdict(list)
hoursPerItem = defaultdict(list)
hourDict = {}
yearDict = {}
ave_hour = 0

for d in dataTrain:
  usersPerItem[d['gameID']].add(d['userID'])
  itemsPerUser[d['userID']].add(d['gameID'])
  hoursPerUser[d['userID']].append(d['hours_transformed'])
  hoursPerItem[d['gameID']].append(d['hours_transformed'])
  hourDict[(d['userID'],d['gameID'])] = d['hours_transformed']
  yearDict[(d['userID'],d['gameID'])] = int(d['date'][:4])
  ave_hour += d['hours_transformed']

for d in dataTest:
  yearDict[(d['userID'],d['gameID'])] = int(d['date'][:4])

ave_hour /= len(dataTrain)

In [332]:
def Jaccard(s1, s2):
  intersection = len(s1.intersection(s2))
  union = len(s1.union(s2))
  jaccard_similarity = intersection / union
  return jaccard_similarity

In [333]:
def mostSimilar(i, N):
  js = []
  id = []
  for key, value_set in itemsPerUser.items():
    id.append(key)
    js.append(Jaccard(value_set, itemsPerUser[i]))
  sorted_js = sorted(enumerate(js), key=lambda x: x[1], reverse=True)
  return sorted_js[1][1], sorted_js[N][1]

In [334]:
first, tenth = mostSimilar(dataTrain[0]['userID'], 10)

In [335]:
answers['Q7'] = [first, tenth]

In [338]:
assertFloatList(answers['Q7'], 2)

In [ ]:
### Question 8

In [499]:
def rU(u,i):
  if len(hoursPerItem[i]) == 0 or len(hoursPerUser[u]) == 0:
    return ave_hour

  sum1 = 0
  sum2 = 0
  for v in usersPerItem[i]:
    sum1 += hourDict[(v,i)] * Jaccard(itemsPerUser[u], itemsPerUser[v])
    sum2 += Jaccard(itemsPerUser[u], itemsPerUser[v])
  if sum2 == 0:
    return ave_hour
  else:
    return sum1/sum2

In [500]:
hours_pred = []
hours = []
for d in dataTest:
  hours_pred.append(rU(d['userID'],d['gameID']))
  hours.append(d['hours_transformed'])
MSEU = mean_squared_error(hours_pred,hours)

In [521]:
def rI(u,i):
  if len(hoursPerItem[i]) == 0 or len(hoursPerUser[u]) == 0:
    return ave_hour

  sum1 = 0
  sum2 = 0
  for j in itemsPerUser[u]:
    sum1 += hourDict[(u,j)] * Jaccard(usersPerItem[i], usersPerItem[j])
    sum2 += Jaccard(usersPerItem[i], usersPerItem[j])
  if sum2 == 0:
    return ave_hour
  else:
    return sum1/sum2

In [522]:
hours_pred = []
hours = []
for d in dataTest:
  hours_pred.append(rI(d['userID'],d['gameID']))
  hours.append(d['hours_transformed'])
MSEI = mean_squared_error(hours_pred,hours)

In [523]:
answers['Q8'] = [MSEU, MSEI]

In [490]:
assertFloatList(answers['Q8'], 2)

In [ ]:
### Question 9

In [491]:
def r(u,i):
  if len(hoursPerItem[i]) == 0 and len(hoursPerUser[u]) == 0:
    return ave_hour

  sum1 = 0
  sum2 = 0
  for v in usersPerItem[i]:
    sum1 += hourDict[(v,i)] * Jaccard(itemsPerUser[u], itemsPerUser[v]) * (math.e ** (-abs(yearDict[(u,i)]-yearDict[v,i])))
    sum2 += Jaccard(itemsPerUser[u], itemsPerUser[v]) * (math.e ** (-abs(yearDict[(u,i)]-yearDict[v,i])))
  if sum2 == 0:
    return ave_hour
  else:
    return sum1/sum2

In [492]:
hours_pred = []
hours = []
for d in dataTest:
  hours_pred.append(r(d['userID'],d['gameID']))
  hours.append(d['hours_transformed'])
MSE9 = mean_squared_error(hours_pred,hours)

In [493]:
answers['Q9'] = MSE9

In [376]:
assertFloat(answers['Q9'])

In [377]:
if "float" in str(answers) or "int" in str(answers):
    print("it seems that some of your answers are not native python ints/floats;")
    print("the autograder will not be able to read your solution unless you convert them to ints/floats")

In [398]:
f = open("answers_midterm.txt", 'w')
f.write(str(answers) + '\n')
f.close()